In [1]:
import pandas as pd
import numpy as np


### 데이터 불러오기

In [ ]:
df = pd.read_csv('../resources/processed/v4/train_20241104.csv')

In [3]:
df

,ID,text,target,target_name,source
0,ynat-v1_train_00000,"정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)",4,IT과학,noisy
1,ynat-v1_train_00000,"정부, KT 주파수 미사용에 강력한 제재 조치",4,IT과학,noisy
2,ynat-v1_train_00000,"KT, 주파수 미사용으로 이용기간 2년 단축 처분",4,IT과학,noisy
3,ynat-v1_train_00000,정 파 미사 KT 이용기간 단 분종 보,4,IT과학,noisy
4,ynat-v1_train_00000,"KT, 주파수 미사용으로 인한 사용 기간의 2년 단축.",4,IT과학,noisy
...,...,...,...,...,...
11609,ynat-v1_train_02797,C 버전을 업데이트한 후 텔레그램과 같은 메신저가 2시간 동안 다운되었는데 원인은 ...,4,IT과학,noisy
11610,ynat-v1_train_02797,텔레그램 및 기타 암호화 메신저는 아시아에서 2시간 동안 다운되었습니다... 사용자 혼란,4,IT과학,noisy
11611,ynat-v1_train_02797,"C버전 업데이트 후 텔레그램 등 메신저 2시간 장애 발생, 원인은?",4,IT과학,noisy
11612,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,1,스포츠,not_noisy


### sentence 임베딩

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("dragonkue/bge-m3-ko")

In [7]:
sentences = [
    '수급권자 중 근로 능력이 없는 임산부는 몇 종에 해당하니?',
    '내년부터 저소득층 1세 미만 아동의 \n의료비 부담이 더 낮아진다!\n의료급여제도 개요\n□ (목적) 생활유지 능력이 없거나 생활이 어려운 국민들에게 발생하는 질병, 부상, 출산 등에 대해 국가가 의료서비스 제공\n□ (지원대상) 국민기초생활보장 수급권자, 타 법에 의한 수급권자 등\n\n| 구분 | 국민기초생활보장법에 의한 수급권자 | 국민기초생활보장법 이외의 타 법에 의한 수급권자 |\n| --- | --- | --- |\n| 1종 | ○ 국민기초생활보장 수급권자 중 근로능력이 없는 자만으로 구성된 가구 - 18세 미만, 65세 이상 - 4급 이내 장애인 - 임산부, 병역의무이행자 등 | ○ 이재민(재해구호법) ○ 의상자 및 의사자의 유족○ 국내 입양된 18세 미만 아동○ 국가유공자 및 그 유족․가족○ 국가무형문화재 보유자 및 그 가족○ 새터민(북한이탈주민)과 그 가족○ 5․18 민주화운동 관련자 및 그 유가족○ 노숙인 ※ 행려환자 (의료급여법 시행령) |\n| 2종 | ○ 국민기초생활보장 수급권자 중 근로능력이 있는 가구 | - |\n',
    '이어 이날 오후 1시30분부터 열릴 예정이던 스노보드 여자 슬로프스타일 예선 경기는 연기를 거듭하다 취소됐다. 조직위는 예선 없이 다음 날 결선에서 참가자 27명이 한번에 경기해 순위를 가리기로 했다.',
]
embeddings = model.encode(sentences)
print(embeddings.shape)

(3, 1024)


In [11]:
sentences = df['text'].tolist()

In [12]:
len(sentences)

11614

In [13]:
embeddings = model.encode(sentences)
print(embeddings.shape)

(11614, 1024)


In [14]:
embeddings

array([[-0.05443421,  0.04084773, -0.01338731, ..., -0.03000877,
        -0.03458807, -0.04642584],
       [-0.01035193,  0.02598784, -0.02697789, ..., -0.0486051 ,
        -0.06027713,  0.0067699 ],
       [-0.04928539,  0.047268  , -0.00631128, ..., -0.03470273,
        -0.01927795, -0.03544687],
       ...,
       [-0.00113946, -0.00393194, -0.01541031, ..., -0.00628853,
         0.00326862, -0.01588001],
       [-0.00372862, -0.03413697, -0.02908899, ..., -0.01000832,
        -0.01245608, -0.05036394],
       [ 0.0358014 ,  0.0158757 ,  0.02738611, ...,  0.00015073,
        -0.01764536,  0.03216164]], dtype=float32)

In [ ]:
np.save('embeddings.npy', embeddings)

### 임베딩 벡터 불러와서 similarity 계산

In [4]:
embeddings = np.load('embeddings.npy')

In [5]:
embeddings.shape

(11614, 1024)

In [7]:
df_noisy = df[df['source'] == 'noisy']

In [8]:
topic_similarity_dic = {}
for _, group in df_noisy.groupby('target'):
    target = group['target'].tolist()[0]
    idx = group.index.tolist()
    embeddings_idx = embeddings[idx]
    target_embedding = np.mean(embeddings_idx, axis=0)
    topic_similarity_dic[target] = target_embedding

In [19]:
df['target'].value_counts()

target
6    1726
1    1725
2    1705
0    1660
3    1625
4    1615
5    1558
Name: count, dtype: int64

In [37]:
sim_result = []
for _, group in df.groupby('target'):
    target = group['target'].tolist()[0]
    print(target)
    target_embedding = topic_similarity_dic[target]
    idx = group.index.tolist()
    embedding_lst = embeddings[idx]
    target_similarity = embedding_lst @ target_embedding.T
    print(len(target_similarity))
    print(target_similarity.max())
    print(target_similarity.min())
    for i, s in zip(idx, target_similarity):
        sim_result.append([i, round(float(s), 5)])
    print()
    

0
1660
0.2531471
0.11968984

1
1725
0.29185572
0.100463316

2
1705
0.31173044
0.13385652

3
1625
0.2805401
0.12618788

4
1615
0.33742547
0.13157488

5
1558
0.32789928
0.112449445

6
1726
0.27827552
0.13228665



In [42]:
df = df.reset_index()

In [43]:
df

,index,ID,text,target,target_name,source
0,0,ynat-v1_train_00000,"정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)",4,IT과학,noisy
1,1,ynat-v1_train_00000,"정부, KT 주파수 미사용에 강력한 제재 조치",4,IT과학,noisy
2,2,ynat-v1_train_00000,"KT, 주파수 미사용으로 이용기간 2년 단축 처분",4,IT과학,noisy
3,3,ynat-v1_train_00000,정 파 미사 KT 이용기간 단 분종 보,4,IT과학,noisy
4,4,ynat-v1_train_00000,"KT, 주파수 미사용으로 인한 사용 기간의 2년 단축.",4,IT과학,noisy
...,...,...,...,...,...,...
11609,11609,ynat-v1_train_02797,C 버전을 업데이트한 후 텔레그램과 같은 메신저가 2시간 동안 다운되었는데 원인은 ...,4,IT과학,noisy
11610,11610,ynat-v1_train_02797,텔레그램 및 기타 암호화 메신저는 아시아에서 2시간 동안 다운되었습니다... 사용자 혼란,4,IT과학,noisy
11611,11611,ynat-v1_train_02797,"C버전 업데이트 후 텔레그램 등 메신저 2시간 장애 발생, 원인은?",4,IT과학,noisy
11612,11612,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,1,스포츠,not_noisy


In [45]:
df_sim = pd.DataFrame(sim_result, columns=['index', 'similarity'])

In [54]:
df = pd.merge(df, df_sim, how = 'left', on='index')

### 히스토그램 및 이상치 확인하여 제거

In [ ]:
import matplotlib.pyplot as plt

def find_outliers(data, target_name=None):
    # 리스트를 NumPy 배열로 변환
    arr = np.array(data)
    
    # 사분위수 계산
    Q1 = np.percentile(arr, 25)
    Q3 = np.percentile(arr, 75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 상한과 하한 경계 계산
    # if target_name == '경제':
    lower_bound = Q1 - 0.5 * IQR
    upper_bound = Q3 + 0.5 * IQR
    # else:
        # lower_bound = Q1 - 0.5 * IQR
        # upper_bound = Q3 + 1 * IQR
    
    
    # 이상치 찾기
    outliers = arr[(arr < lower_bound) | (arr > upper_bound)]
    
    return lower_bound, upper_bound, outliers

In [ ]:
remove_idx = []
for _, group in df.groupby('target'):
    target_name = group['target_name'].tolist()[0]
    print(target_name)
    sim_lst = group['similarity'].tolist()
    lower_bound, upper_bound, outliers = find_outliers(sim_lst, target_name)
    print(lower_bound, upper_bound)
    lower_text_lst = group[group['similarity'] <= lower_bound]['text'].tolist()
    lower_idx_lst = group[group['similarity'] <= lower_bound].index.tolist()
    remove_idx.extend(lower_idx_lst)
    print(len(lower_text_lst), lower_text_lst)
    upper_text_lst = group[group['similarity'] >= upper_bound]['text'].tolist()
    upper_idx_lst = group[group['similarity'] >= upper_bound].index.tolist()
    upper_idx_lst_remove = np.random.choice(group.index, size=max(int(len(upper_idx_lst) * 0.95), 1) , replace=False)
    remove_idx.extend(upper_idx_lst_remove)
    print(len(upper_text_lst), upper_text_lst)
    # bins = np.arange(0.1, max(sim_lst), 0.005)
    # plt.figure(figsize=(12, 6))
    # plt.hist(sim_lst, bins=bins[:-1], edgecolor='black', alpha=0.7)
    # plt.title(target_name)
    print()
    

생활문화
0.16072874999999998 0.21452375
150 ['한국민족의 뿌리가 북방 천년 전 한반도서 자생적으로 형성', '근육조직 생성에 필수적인 열쇠 발견', '공룡 파충류보다는 타조 매 등 새에 가깝다', '폭스바겐 아스턴 마틴 스파라 국내 생산 500대', '아스턴 마틴은 한국에서 첫 번째 4P 모델 500대를 생산했습니다.', '본카 애스턴마틴 스파라 국내 500대 생산', '2023년 수능, 전국적으로 안정적 진행...학생들의 건강과 안전 최우선', '수능 6월 모의평가 문제 유출 사건, 교사 1명 구속', '라인서점이 양상동로와 교명평에 진출해 시장점유율을 늘리고 있다.', '서울은 아침 기온이 섭씨 1도인데 낮에는 온화한 날씨가 예상된다.', '한국 연극인 13명, 일본에서 독창적 작품 발표', '산업 현장 안전 문제, 재발 방지 대책 논의 필요', '러시아는 11시 인도 시간에 드론 공격에 대한 대응책을 발표했다.', '인도군 오전 11시 러시아 드론 공격 대응 조치 발표', '기상청은 0시부터 경주 지역에 대해 폭설 경보를 발령합니다.', '9.1도 이하 연천...오후부터 경상북도 지역에서 주로 눈이 내리는 날씨가 예상된다.', '연천, 영하 9.1도까지 떨어져...북부 지역 추위 주의', '최규식 내무장관이 오는 6일 방화대책을 발표한다.', '리J, 인격 교육의 중요성 역설...P과만중KA 독불군형 비판', '리J, 인격적 소양 강조...P과만중KA 독불군형 반대', '성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 대한 역설, 성격 교육의 중요성에 

In [142]:
len(remove_idx)

2103

In [143]:
df_f = df[~df['index'].isin(remove_idx)]

In [144]:
df_f = df_f.reset_index(drop=True)

In [145]:
df_f['target'].value_counts()

target
2    1486
6    1464
0    1380
1    1372
3    1351
4    1322
5    1218
Name: count, dtype: int64

In [147]:
df_f.to_csv('../resources/processed/v4/train_20241104_filtering.csv', encoding='utf-8-sig', index=0)

In [148]:
df_f = pd.read_csv('../resources/processed/v4/train_20241104_filtering.csv')